In [1]:
import fff
from hexdump import hexdump as hd

disk = fff.DiskImage('/Users/xin/practice1.dd.zip')

disk.volume

  #  Slot      Start  End       Length  Description
---  ------  -------  ------  --------  ----------------------------------
  1  0             0  0              1  Master Boot Record
  2  0:-           1  31            30  Unallocated
  3  0:0          32  30719      30688  Linux native partition (0x83)
  4  0:1       30720  98303      67584  Linux swap (0x82)
  5  0:2       98304  208895    110592  Linux native partition (0x83)
  6  0:-      208896  212991      4095  Unallocated
  7  0:3      212992  417791    204800  DOS 3.3+ Extended Partition (0x5)
  8  1        212992  212992         1  Extended Boot Record
  9  1:-      212993  213023        30  Unallocated
 10  1:0      213024  286719     73696  Linux native partition (0x83)
 11  1:1      286720  417791    131072  DOS 3.3+ Extended Partition (0x5)
 12  2        286720  286720         1  Extended Boot Record
 13  2:-      286721  286751        30  Unallocated
 14  2:0      286752  417791    131040  Linux native partition (0x83

In [2]:
print(disk.volume[4])

  #  Slot      Start    End    Length  Description        CHS
---  ------  -------  -----  --------  -----------------  ---------------------
  4  0:1       30720  98303     67584  Linux swap (0x82)  CHS 15/0/1 - 47/63/32


In [3]:
fat = disk.volume[10].filesystem
print(fat.boot_sector)

In [4]:
print(disk.volume[3])
hd(disk.volume[14].sectors[0][0:0x20])

  #  Slot      Start    End    Length  Description                    CHS
---  ------  -------  -----  --------  -----------------------------  --------------------
  3  0:0          32  30719     30688  Linux native partition (0x83)  CHS 0/1/1 - 14/63/32
00000000: EB 52 90 4E 54 46 53 20  20 20 20 00 02 02 00 00  .R.NTFS    .....
00000010: 00 00 00 00 00 F8 00 00  20 00 40 00 20 60 04 00  ........ .@. `..


In [5]:
ntfs = disk.volume[14].filesystem
ntfs.boot_sector

Field                            Value
-------------------------------  --------------------------
JMP                              eb5290
Signature                        NTFS    (4e54465320202020)
Bytes Per Sector                 512
Sectors Per Cluster              2
...                              ...
Media Descriptor                 248
Sectors Per Track                32
Number of Heads                  64
Hidden Sectors                   286752
...                              ...
Total Sectors                    131039
$MFT Cluster Number              16
$MFTMirr Cluster Number          32759
Cluster Per File Record Segment  1
...                              ...
Cluster Per Index Buffer         4
...                              ...
Volume Serial Number             7829755030752268642
...                              ...
Bootstrap Code                   fa33c08ed0bc007cfb68
Marker                           55aa

In [6]:
print(ntfs.mft.mft_entry)
[print(a) for a in ntfs.mft.mft_entry.attrs]

320 0x12 [] 251 16
392 0x11 [] 1 8
Field                          Value
-----------------------------  --------------
Signature                      FILE(46494c45)
Offset to Fixup                48
Entry Count in Fixup Array     3
$LogFile Sequence Number       0
Sequence Number                1
Link Count                     1
Attribute Offset               56
Flags                          0b1
Used Size of MFT Entry         408
Allocated Size of MFT Entry    1024
File Reference to Base Record  0
Next Attribute ID              4
#attributes                    4
Field                   Value
----------------------  -------------------------
Type ID                 16 ($StandardInformation)
Name                    b''
Size                    96
Non-Resident Flag       False (Resident)
Name Length             0
Name Offset             24
Flags                   0b0
Attribute ID            0
Attribute Length        3577643008
Created                 116444736000000000
Modified            

[None, None, None, None]

In [7]:
print(ntfs.mft.attr(type_id='$DATA'))

Field                  Value
---------------------  -------------------
Type ID                128 ($DATA)
Name                   b''
Size                   72
Non-Resident Flag      True (Non-Resident)
Name Length            0
Name Offset            64
Flags                  0b0
Attribute ID           1
Attribute Length       268499730
Starting VCN           0
Last VCN               250
DataRuns Offset        64
Compression Unit Size  0
Allocated Size         257024
Actual Size            257024
Compressed Size        257024
#Data Runs             1
Data                   None


In [9]:
print(ntfs.mft.attr(type_id='$DATA').data_runs)
hd(next(ntfs.mft.read2(bsize=1024, count=251)))

[DataRun(length=251, offset=16)]
00000000: EB 52 90 4E 54 46 53 20  20 20 20 00 02 02 00 00  .R.NTFS    .....
00000010: 00 00 00 00 00 F8 00 00  20 00 40 00 20 60 04 00  ........ .@. `..
00000020: 00 00 00 00 80 00 80 00  DF FF 01 00 00 00 00 00  ................
00000030: 10 00 00 00 00 00 00 00  F7 7F 00 00 00 00 00 00  ................
00000040: 01 00 00 00 04 00 00 00  62 11 51 76 B7 E0 A8 6C  ........b.Qv...l
00000050: 00 00 00 00 FA 33 C0 8E  D0 BC 00 7C FB 68 C0 07  .....3.....|.h..
00000060: 1F 1E 68 66 00 CB 88 16  0E 00 66 81 3E 03 00 4E  ..hf......f.>..N
00000070: 54 46 53 75 15 B4 41 BB  AA 55 CD 13 72 0C 81 FB  TFSu..A..U..r...
00000080: 55 AA 75 06 F7 C1 01 00  75 03 E9 D2 00 1E 83 EC  U.u.....u.......
00000090: 18 68 1A 00 B4 48 8A 16  0E 00 8B F4 16 1F CD 13  .h...H..........
000000A0: 9F 83 C4 18 9E 58 1F 72  E1 3B 06 0B 00 75 DB A3  .....X.r.;...u..
000000B0: 0F 00 C1 2E 0F 00 04 1E  5A 33 DB B9 00 20 2B C8  ........Z3... +.
000000C0: 66 FF 06 11 00 03 16 0F  00 8E C2

00005670: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00005680: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00005690: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000056A0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000056B0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000056C0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000056D0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000056E0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000056F0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00005700: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00005710: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00005720: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00005730: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................

000094E0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000094F0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00009500: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00009510: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00009520: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00009530: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00009540: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00009550: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00009560: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00009570: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00009580: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00009590: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000095A0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................

0000F2A0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
0000F2B0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
0000F2C0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
0000F2D0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
0000F2E0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
0000F2F0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
0000F300: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
0000F310: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
0000F320: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
0000F330: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
0000F340: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
0000F350: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
0000F360: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................

00015050: 80 14 DD C9 B8 95 C9 01  80 14 DD C9 B8 95 C9 01  ................
00015060: 80 14 DD C9 B8 95 C9 01  80 14 DD C9 B8 95 C9 01  ................
00015070: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00015080: 20 00 00 00 48 00 00 00  01 00 40 00 00 00 04 00   ...H.....@.....
00015090: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000150A0: 40 00 00 00 00 00 00 00  00 04 00 00 00 00 00 00  @...............
000150B0: D8 00 00 00 00 00 00 00  D8 00 00 00 00 00 00 00  ................
000150C0: 21 01 A0 31 00 00 00 00  50 00 00 00 48 00 00 00  !..1....P...H...
000150D0: 01 00 40 00 00 00 01 00  00 00 00 00 00 00 00 00  ..@.............
000150E0: 00 00 00 00 00 00 00 00  40 00 00 00 00 00 00 00  ........@.......
000150F0: 00 04 00 00 00 00 00 00  50 00 00 00 00 00 00 00  ........P.......
00015100: 50 00 00 00 00 00 00 00  21 01 A2 21 00 00 00 00  P.......!..!....
00015110: A0 00 00 00 50 00 00 00  01 04 40 00 00 00 06 00  ....P.....@.....

...

000305A0: 32 A0 01 E8 E7 00 00 00  FF FF FF FF 00 00 00 00  2...............
000305B0: 00 FF FF FF 00 00 00 00  FF FF FF FF 00 00 00 00  ................
000305C0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000305D0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000305E0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000305F0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 D4 00  ................
00030600: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00030610: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00030620: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00030630: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00030640: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00030650: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00030660: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................

00033D90: F8 53 02 00 00 00 00 00  32 95 00 8A E9 00 00 00  .S......2.......
00033DA0: FF FF FF FF 00 00 00 00  00 FF FF FF 00 00 00 00  ................
00033DB0: FF FF FF FF 00 00 00 00  00 00 00 00 00 00 00 00  ................
00033DC0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00033DD0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00033DE0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00033DF0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 50 00  ..............P.
00033E00: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00033E10: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00033E20: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00033E30: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00033E40: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00033E50: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................

00036350: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00036360: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00036370: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00036380: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00036390: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000363A0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000363B0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000363C0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000363D0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000363E0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000363F0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 E6 08  ................
00036400: 46 49 4C 45 30 00 03 00  00 00 00 00 00 00 00 00  FILE0...........
00036410: 01 00 01 00 38 00 01 00  A0 01 00 00 00 04 00 00  ....8...........

00039E60: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00039E70: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00039E80: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00039E90: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00039EA0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00039EB0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00039EC0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00039ED0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00039EE0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00039EF0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00039F00: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00039F10: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00039F20: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................

0003C100: 00 00 18 00 00 00 01 00  50 00 00 00 18 00 00 00  ........P.......
0003C110: 01 00 04 80 14 00 00 00  24 00 00 00 00 00 00 00  ........$.......
0003C120: 34 00 00 00 01 02 00 00  00 00 00 05 20 00 00 00  4........... ...
0003C130: 20 02 00 00 01 02 00 00  00 00 00 05 20 00 00 00   ........... ...
0003C140: 20 02 00 00 02 00 1C 00  01 00 00 00 00 03 14 00   ...............
0003C150: FF 01 1F 00 01 01 00 00  00 00 00 01 00 00 00 00  ................
0003C160: 80 00 00 00 48 00 00 00  01 00 40 00 00 00 02 00  ....H.....@.....
0003C170: 00 00 00 00 00 00 00 00  2E 00 00 00 00 00 00 00  ................
0003C180: 40 00 00 00 00 00 00 00  00 BC 00 00 00 00 00 00  @...............
0003C190: AC B9 00 00 00 00 00 00  AC B9 00 00 00 00 00 00  ................
0003C1A0: 21 2F A7 65 00 00 00 00  FF FF FF FF 00 00 00 00  !/.e............
0003C1B0: FF FF FF FF 00 00 00 00  00 00 00 00 00 00 00 00  ................
0003C1C0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................